# Installation


In [1]:
%%capture
import os, re, random
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

# Dataset


In [2]:
from datasets import load_dataset

dataset = load_dataset("INK-USC/riddle_sense", trust_remote_code=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

riddle_sense.py: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/3510 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1021 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1184 [00:00<?, ? examples/s]

In [3]:
print(dataset['train']['question'][0])
print(dataset['train']['answerKey'][0])
print(dataset['train']['choices'][0]['text'][4])

A man is incarcerated in prison, and as his punishment he has to carry a one tonne bag of sand backwards and forwards across a field the size of a football pitch.  What is the one thing he can put in it to make it lighter?
E
hole


In [4]:
def getExample():
  rndm = random.randint(1, 2000)
  shot = "Riddle: " + dataset['train']['question'][rndm] + "\n"
  ans = dataset['train']['answerKey'][rndm]
  ansKey = {'A':0,'B':1,'C':2,'D':3,'E':4}
  shot += "Answer: " + dataset['train']['choices'][rndm]['text'][ansKey[ans]]
  return shot

print(getExample())

Riddle: He was afraid to go home because of the man in the mask.    Who is the man in the mask?
Answer: man in mask is catcher


# Model

In [5]:
#code from https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_v0.3_(7B)-Conversational.ipynb

from unsloth import FastLanguageModel
import torch
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3",
    max_seq_length = 4096, # Choose any!
    dtype = None, # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit = True, # Use 4bit quantization to reduce memory usage. Can be False.
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


==((====))==  Unsloth 2025.12.4: Fast Mistral patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [6]:
%%capture
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "mistral",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# FINETUNING

Function to prompt the model

In [7]:
def promptMinstral(myprompt):
  messages = [{"from":"human", "value": myprompt}]
  inputs = tokenizer.apply_chat_template(
      messages,
      tokenize = True,
      add_generation_prompt = True, # Must add for generation
      return_tensors = "pt",
  ).to("cuda")
  outputs = model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True)
  return tokenizer.batch_decode(outputs)[0]

# Regex

In [8]:
import re

def decodeOutput(riddle):
  decoded = re.split(r"\[/INST\]",riddle)
  decoded = decoded[1] # cuts off prompt

  riddle = re.search("[r|R]iddle: (.*)", decoded)
  if not riddle:
    riddlesplit = decoded.split('\n')
    riddle = riddlesplit[0] if len(riddlesplit[0]) > 1 else riddlesplit[1]
  else:
    riddle = riddle[0]

  answer = re.search("[a|A]nswer: (.*)", decoded)
  if not answer:
    answer = "None"
  else:
    answer = answer[0]

  return riddle, answer

In [9]:
myInput = """Your job is to generate a riddle given another riddle. Only send the riddle and answer. Following is an example of a riddle you should emulate. Example: """ + getExample()

rawOutput = promptMinstral(myInput)

print(rawOutput)
print("---------------------------\n")

riddle, answer = decodeOutput(rawOutput)
print(riddle, '\n',answer)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<s>[INST] Your job is to generate a riddle given another riddle. Only send the riddle and answer. Following is an example of a riddle you should emulate. Example: Riddle: What is often returned but never borrowed?
Answer: thanks and compliments [/INST]

Riddle: What is the only thing that you can break, even if you never touch it?
Answer: a promise

Riddle: What is the only thing that you can break, even if you never touch it?
Answer: a promise

Riddle: What is the only thing that you can break, even if you never touch it?
Answer: a promise

Riddle: What is the only thing that you can break, even if you never touch it?
Answer: a promise

Riddle: What is the only thing that you can break, even if you never touch it?
Answer: a promise

Riddle: What is the only thing that you can break, even if you never touch it?
Answer: a promise

Riddle: What is the only thing that you can break, even if you never touch it?
Answer: a promise

Riddle: What is the only thing that you can break, even if y

Location nodes and map

In [ ]:
class Location:
  def __init__(self, block = False):
    self.blocked = block
    # room connections
    self.north = None
    self.east = None
    self.west = None
    self.south = None
    # list of items - there's one case where there can be more than one item but its useful
    self.items = []
    # description of the room
    self.desc = None
    # bool as to whether a riddle can occur in the room
    self.riddle = False
    # message that the parser reads if the way is blocked.
    self.blocked_msg = None

In [ ]:
# Basicest map
start = Location()
hallway = Location()
candleRoom = Location()
bugRoom = Location(True)
escapeRoom = Location()

# Joining up the locations
start.east = hallway

hallway.east = candleRoom
hallway.west = start
hallway.south = bugRoom

candleRoom.west = hallway

bugRoom.north = hallway
bugRoom.south = escapeRoom

escapeRoom.north = bugRoom

# Set items in the rooms
candleRoom.items.append("candle")
hallway.items.append("rocks")

# Establishes the rooms the riddles will occur in
start.riddle = True
candleRoom.riddle = True
escapeRoom.riddle = True

# Set descriptions for the room
start.desc = """The room you first woke up in. The air is moist and the smell of mildew fills your senses. There is an opening to the east."""

hallway.desc = """A room that connects to three others.
It's dim but you can faintly see a light from the room to the west.
The room to the south is loud. It sounds like something is alive down there.
The room to the east is where you woke up."""

candleRoom.desc = """A surprisingly bright room for the tomb. Its lined with many fires, and the walls seem gilded with gold. You're unfortunately unable to collect any.
The room to the west is the hallway."""

bugRoom.desc = """The sound grows louder and louder until you realize what it is. Bugs. The floor is covered in them and they all seem aggressive.
The room to the north is the hallway.
To the south seems to be your escape"""

escapeRoom.desc = """You can just barely see the daylight! You're nearly freed.
'escape' to free yourself!
"""

# Set block messages to the room
bugRoom.blocked_msg = "The room is teeming with bugs you're unable to take so much as a step inside without them crawling on you, biting your legs up."
escapeRoom.blocked_msg = "You cannot leave. The Sphinx won't let you"

Parser class

In [ ]:
class Parser:
  def __init__(self, debug = False):
    self.lives = 3
    self.location = start
    self.inventory = []
    self.level = 0
    self.victory = False
    self.debug = debug

  def ask(self):
    # promtpts the LLM to output a new riddle in the form of a json file
    # that includes the riddle, a hint and the answer. The function runs a nested loop that
    # checks users answer to give you multiple tries.
    #
    #
    # inputs : None
    # output : True if the question was correctly answered, False if not.

    # Input prompt
    myInput = """Your job is to generate a riddle given another riddle. Only send the riddle and answer. Following is an example of a riddle you should emulate. Example: """ + getExample()
    minimenu = """You may 'give up' and get another question\nYour answer: """

    try:
      # Generate text
      modelOutput = promptMinstral(myInput)

      # Decode and print the output
      riddle, answer = decodeOutput(rawOutput)
    except Exception as e:
      print("Error when generating riddle. Asking again.")
      self.ask() # start over if the model crashed

    riddle_loop = True
    while riddle_loop and self.lives > 0: # different from the gameloop, riddle loop, consists only of the sphinx's question and answering.
      print(riddle)
      if self.debug:
        print("======================================================")
        print("BACKEND DETAILS: ")
        print(modelOutput)
        print(riddle)
        print(answer)
        print("======================================================")

      response = input(minimenu)
      response = response.lower()
      responseList = response.split(" ")


      if self.answer == "None":
        print("The sphinx has a stroke and forgot to think of an answer to its own riddle. It's your lucky day")
        return True
      if responseList[0] == 'give' and responseList[1] == 'up':
        print("Try as you might, you can't figure this one out. The sphinx claws at you for your incompetence, costing you dearly.")
        self.lives -= 1
        return False
      elif responseList[0] == 'what':
        print("The sphinx's old age is clearly getting to it. You fail to understand the question and it gets really embarassed\nIt leaves you for now...")
        return True
      if self.answer(response, answer):
        print("That was correct.")
        print("The sphinx looks annoyed but it sulks off in defeat.")
        self.location.riddle = False
        print("From nowhere " + answer + "drops to the floor from above. You can't see where it came from but it looks useful")
        self.location.items.append(answer)
        return True
      else:
        print("The sphinx makes a noise like laughter. You're wrong. One step closer to being sealed inside forever.")
        self.lives -= 1

    # case where you failed the riddle and died.
    print("The sphinx deems you a lost clause. The beast lunges at you and in an instant you're devoured.")
    return False

    def answer(self, response, answer):
      # answer - compares the response given to the answer the model provided.
      #
      # inputs: user response and the answer given by the llm
      # returns: True or False
      # True - the answer is correct, it passes the threshold
      # False - the answer is incorrect.


      # tokenize
      resp_tokens = response.lower().split()
      ans_tokens = answer.lower().split()

      # build frequency dicts
      resp_vec = {}
      for w in resp_tokens:
        resp_vec[w] = resp_vec.get(w, 0) + 1

      ans_vec = {}
      for w in ans_tokens:
        ans_vec[w] = ans_vec.get(w, 0) + 1

      # vocabulary
      all_words = set(resp_vec.keys()) | set(ans_vec.keys())

      # numeric vectors
      v1 = [resp_vec.get(w, 0) for w in all_words]
      v2 = [ans_vec.get(w, 0) for w in all_words]

      # cosine similarity
      dot = sum(a * b for a, b in zip(v1, v2))
      mag1 = math.sqrt(sum(a * a for a in v1))
      mag2 = math.sqrt(sum(b * b for b in v2))

      cosine_sim = 0 if (mag1 == 0 or mag2 == 0) else dot / (mag1 * mag2)

      # threshold check
      return True if cosine_sim >= 0.4 else False

      return True if eval >= 0.4 else False # thresholds the % match of the

  def prologue(self):
    # Prologue -
    # returns nothing but easier to navigate than multiple strings

    # Updates things that should change during the game
    bugRoom.desc = """The sound grows louder and louder until you realize what it is. Bugs. The floor is covered in them and they all seem aggressive.
    The room to the north is the hallway.
    To the south seems to be your escape"""
    start.riddle = True
    candleRoom.riddle = True
    escapeRoom.riddle = True


    print("""Welcome to the game. Placeholder text for the openning.
    More story stuff to come.
    Lot of lines
    I kinda hope i can slow this down between each line but probably not
    help for list of commands.""") # more to come than this but its a placeholder

  def game(self):
    # Game - Provides the game loop and main functionality of the game.
    # The game ends when the player quits, runs out of lives or they
    # correctly respond to three riddles.

    playing = True

    self.prologue()

    while(playing and self.lives > 0 and not self.victory): # GAMELOOP

      # Room message.
      print(self.location.desc)
      if(self.location.blocked):
        print("You aren't be able to go any further south at this time.")
      if self.location.riddle:
        print("The sphinx moves in front of the door. You must 'play' her game to pass.")

      # Takes user response and splits it before going through commands
      response = input("")
      response = response.lower()
      responseList = response.split(" ")

      # the big switch case controlling our game.
      if(responseList[0] == "q" or responseList[0] == "quit"):
        playing = False # User ends the game on their own, it ends here

      # plays - enters the riddle.
      elif(responseList[0] == 'play'):
        if self.location.riddle:
          if self.ask():
            self.level+=1
            print("Your chest swells with pride over your victory.")
          else:
            print("You failed the sphinx's riddle!")
        else:
          print("The sphinx is not in this room.")

      # Directional controls for the location system.
      elif(responseList[0] == "n" or responseList[0] == "north"):
          if self.location.north:
            if not self.location.riddle or self.debug:
                self.location = self.location.north
            else:
              print("The sphinx has you trapped in this room until you can answer her riddle!")
          else: print("There is no north exit.")
      elif(responseList[0] == "e" or responseList[0] == "east"):
          if self.location.east:
            if not self.location.riddle or self.debug:
              self.location = self.location.east
            else:
              print("The sphinx has you trapped in this room until you can answer her riddle!")
          else: print("There is no east exit.")
      elif(responseList[0] == "w" or responseList[0] == "west"):
          if self.location.west:
            if not self.location.riddle or self.debug:
              self.location = self.location.west
            else:
              print("The sphinx has you trapped in this room until you can answer her riddle!")
          else: print("There is no west exit.")
      elif(responseList[0] == "s" or responseList[0] == "south"):
        if self.location.south:
          if not self.location.blocked: # Progress will only ever go south, so we only need to check if its blocked here
              if not self.location.riddle or self.debug: # checks if in debug mode or if you've finished the riddle
               self.location = self.location.south
              else:
                print("The sphinx has you trapped in this room until you can answer her riddle!")
          else:
            print(self.location.blocked_msg)
        else: print("There is no south exit.")

      # give up command - going to phase out as we'll move the riddles to the ask function
      elif(responseList[0] == "give" and responseList[1] == "up"):
         print("You succumb to despair, losing a life")
         self.lives -= 1

      # why - why'd you get that last answer right?
      elif(responseList[0] == "why"):
        if self.explanation:
          print(self.explanation)
        else:
          print("Why what?")

      # look around - checks all exits
      elif(responseList[0] == "look"):
        if self.location.north:
          print("There is an exit to the north")
        if self.location.east:
          print("There is an exit to the east")
        if self.location.west:
          print("There is an exit to the west")
        if self.location.south:
          print("There is an exit to the south")

        print(self.location.desc)
        if(self.location.blocked):
          print("You are won't be able to go south. Something is blocking the way.")
        if self.location.riddle:
          print("The sphinx is in this room, stopping you from leaving until you best it at its own game.")
        if (self.location.items): # If there are items they're listed
          print("You see something useful: ")
          for item in self.location.items:
            print("\t a " + item)

      # get - it allows players to
      elif(responseList[0] == "get"):
        if self.location.items:
          self.inventory = self.inventory + self.location.items # conjoins the lists
          self.location.items = [] # empties out the list of items
        else:
          print("Have you gone mad already? There's nothing there.")

      # light - our main text based puzzle - players light up their candle to unblock the bug room
      elif(responseList[0] == "light" or "fire" in responseList):
        if "candle" or "wood" in self.inventory:
          if self.location == bugRoom:
            if "rocks" in self.inventory:
              print("You start a fire that scares the bugs away. You're now free to continue through this room")
              bugRoom.blocked = False
              bugRoom.desc = "Updated message that no longer has bugs"
              self.inventory.remove("candle")
              self.inventory.remove("rocks")
          else:
            print("There's no reason to do that here.")
        else:
          print("Despite your attempts to start a fire with your mind you have nothing to set ablaze")
      # escape - Win the game if sphinx doesn't have you in for a riddle
      elif(responseList[0] == "escape"):
        if self.location==escapeRoom and not self.location.riddle:
          print("You bust through a nearby wall into the open air! You've defeated the sphinx and now you're free!")
          self.victory = True
        if self.debug:
          print("You win through your amazing cheating skills")
          self.victory = True

      # help gives the controls
      elif(responseList[0] == "help" or responseList[0] == "h"):
        print("Controls :")
        print("\t" +"play - In rooms where the sphinx lurks you can play its game. It won't let you go until you answer correctly.")
        print("\t" +"why - after being asked a riddle and answering successfully you can hear the sphinx's explanation of the riddle.")
        print("\t" + "q/quit - exits out of the game")

        print("\t" +"Directions: ")
        print("\t" + "\t" + "n/north - moves to the room north of this one")
        print("\t" + "\t" + "e/east - moves to the room east of this one")
        print("\t" + "\t" + "s/south - moves to the room south of this one")
        print("\t" + "\t" + "w/west - moves to the room west of this one")

        print("\t" +"look - describes the room's exits, blockages and any items in it. Very useful")
        print("\t" +"inventory - shows you the items you've obtained")
        print("\t" +"get - adds all items in a room to your inventory.")
        print("\t" +"There are more commands related to any items you might find! Try things out!")

      # else - the player misinput
      else:
        print("Not a registered command. Please type 'help' for a list of all controls.")


    # The gameloop has ended.
    if self.victory:
      print("Congratulations! You're free from the sphinx's game and did not meet a fate most gruesome.")
    else:
      if self.lives >= 0:
        print("Your adventure ends here. Your heart comes to a sudden stop.")
      print("The tomb seals shut, dooming you to spend the rest of your short life within this miserable pit.")

    return


In [ ]:
pharohs_game = Parser(True)
pharohs_game.game()

Welcome to the game. Placeholder text for the openning.
    More story stuff to come.
    Lot of lines
    I kinda hope i can slow this down between each line but probably not
    help for list of commands.
The room you woke up in. The air is moist and the smell of mildew fills your lungs. There is an opening to the east.
The sphinx moves in front of the door. You must 'play' her game to pass.
look
There is an exit to the east
The room you woke up in. The air is moist and the smell of mildew fills your lungs. There is an opening to the east.
The sphinx is in this room, stopping you from leaving until you best it at its own game.
The room you woke up in. The air is moist and the smell of mildew fills your lungs. There is an opening to the east.
The sphinx moves in front of the door. You must 'play' her game to pass.
e
A room that connects to three others.
It's dim but you can faintly see a light from the room to the west.
The room to the south is loud. It sounds like something is alive 

KeyboardInterrupt: Interrupted by user